# Data Warehouse Exploration & Hadoop Integration Report

**Step-by-Step Approach**

**1. Initial Data Exploration ("Getting a Feel")**
- Preview the raw file using Excel or a text editor.
- Check structure: Inspect column names, data types, and identify missing or malformed
data.
- Sample rows to understand the kind of information captured.
- Questions to ask:
* Are there date columns? Are they in the correct format?
* Are there categorical vs numerical values?
* Are there empty or corrupted rows?

**2. Loading Dataset into Hadoop**

*Preprocessing:*
* Convert the dataset to clean .csv (if required).
* Upload it to HDFS using: hdfs dfs -put wfpvam_foodprices.csv
/user/yourusername/data/
- Verify upload: hdfs dfs -ls /user/yourusername/data/

In [1]:
#download hadoop
!wget https://downloads.apache.org/hadoop/common/hadoop-3.4.1/hadoop-3.4.1.tar.gz

--2025-05-20 21:28:13--  https://downloads.apache.org/hadoop/common/hadoop-3.4.1/hadoop-3.4.1.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.208.237, 135.181.214.104, 2a01:4f8:10a:39da::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.208.237|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 974002355 (929M) [application/x-gzip]
Saving to: ‘hadoop-3.4.1.tar.gz’

hadoop-3.4.1.tar.gz 100%[===================>] 928.88M  30.3MB/s    in 30s     

2025-05-20 21:28:43 (30.5 MB/s) - ‘hadoop-3.4.1.tar.gz’ saved [974002355/974002355]



In [2]:
#I use the tar command with the -x flag to extract, -z to uncompress,
#-v for verbose output, and -f to specify that we’re extracting from a file
!tar -xzvf hadoop-3.4.1.tar.gz

Streaming output truncated to the last 5000 lines.
hadoop-3.4.1/share/doc/hadoop/api/org/apache/hadoop/fs/cosn/auth/package-summary.html
hadoop-3.4.1/share/doc/hadoop/api/org/apache/hadoop/fs/cosn/auth/package-use.html
hadoop-3.4.1/share/doc/hadoop/api/org/apache/hadoop/fs/cosn/package-frame.html
hadoop-3.4.1/share/doc/hadoop/api/org/apache/hadoop/fs/cosn/package-summary.html
hadoop-3.4.1/share/doc/hadoop/api/org/apache/hadoop/fs/cosn/package-use.html
hadoop-3.4.1/share/doc/hadoop/api/org/apache/hadoop/fs/FSError.html
hadoop-3.4.1/share/doc/hadoop/api/org/apache/hadoop/fs/s3a/
hadoop-3.4.1/share/doc/hadoop/api/org/apache/hadoop/fs/s3a/class-use/
hadoop-3.4.1/share/doc/hadoop/api/org/apache/hadoop/fs/s3a/class-use/Constants.html
hadoop-3.4.1/share/doc/hadoop/api/org/apache/hadoop/fs/s3a/class-use/RemoteFileChangedException.html
hadoop-3.4.1/share/doc/hadoop/api/org/apache/hadoop/fs/s3a/class-use/AWSServiceIOException.html
hadoop-3.4.1/share/doc/hadoop/api/org/apache/hadoop/fs/s3a/class-

In [3]:
#copying the hadoop file to user/local
!cp -r hadoop-3.4.1/ /usr/local/

In [4]:
#Set Home
JAVA_HOME = !readlink -f /usr/bin/java | sed "s:bin/java::"
java_home_text = JAVA_HOME[0]
java_home_text_command = f"${java_home_text[0]}"
!echo export JAVA_HOME=$java_home_text >>/usr/local/hadoop-3.4.1/etc//hadoop/hadoop-env.sh

In [5]:
# Set environment variables
import os
os.environ['HADOOP_HOME']="/usr/local/hadoop-3.4.1"
os.environ['JAVA_HOME']=java_home_text

In [6]:
# Run Hadoop
!/usr/local/hadoop-3.4.1/bin/hadoop

Usage: hadoop [OPTIONS] SUBCOMMAND [SUBCOMMAND OPTIONS]
 or    hadoop [OPTIONS] CLASSNAME [CLASSNAME OPTIONS]
  where CLASSNAME is a user-provided Java class

  OPTIONS is none or any of:

buildpaths                       attempt to add class files from build tree
--config dir                     Hadoop config directory
--debug                          turn on shell script debug mode
--help                           usage information
hostnames list[,of,host,names]   hosts to use in worker mode
hosts filename                   list of hosts to use in worker mode
loglevel level                   set the log4j level for this command
workers                          turn on worker mode

  SUBCOMMAND is one of:


    Admin Commands:

daemonlog     get/set the log level for each daemon

    Client Commands:

archive       create a Hadoop archive
checknative   check native Hadoop and compression libraries availability
classpath     prints the class path needed to get the Hadoop jar and the
  

In [7]:
# Create a folder with HDFS
!/usr/local/hadoop-3.4.1/bin/hadoop fs -mkdir /food_data

In [8]:
# Getting a dataset to analyze with Hadoop
!wget https://data.humdata.org/dataset/4fdcd4dc-5c2f-43af-a1e4-93c9b6539a27/resource/12d7c8e3-eff9-4db0-93b7-726825c4fe9a/download/wfpvam_foodprices.csv

--2025-05-20 21:31:10--  https://data.humdata.org/dataset/4fdcd4dc-5c2f-43af-a1e4-93c9b6539a27/resource/12d7c8e3-eff9-4db0-93b7-726825c4fe9a/download/wfpvam_foodprices.csv
Resolving data.humdata.org (data.humdata.org)... 52.7.154.90, 100.27.77.73, 52.3.104.192
Connecting to data.humdata.org (data.humdata.org)|52.7.154.90|:443... connected.
HTTP request sent, awaiting response... 202 Accepted
Length: 0 [text/html]
Saving to: ‘wfpvam_foodprices.csv’

wfpvam_foodprices.c     [ <=>                ]       0  --.-KB/s    in 0s      

2025-05-20 21:31:10 (0.00 B/s) - ‘wfpvam_foodprices.csv’ saved [0/0]



In [9]:
# Upload the CSV to HDFS
!/usr/local/hadoop-3.4.1/bin/hadoop fs -copyFromLocal wfpvam_foodprices.csv /food_data/

In [10]:
# Verify the file is in HDFS
!/usr/local/hadoop-3.4.1/bin/hadoop fs -ls /food_data

Found 1 items
-rw-r--r--   1 root root          0 2025-05-20 21:31 /food_data/wfpvam_foodprices.csv


In [11]:
# Installing dependencies to use Python
# Dependency to use MapReduce
!pip install mrjob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 20.4 MB/s eta 0:00:00


In [12]:
# Managing temp folder
!/usr/local/hadoop-3.4.1/bin/hadoop fs -mkdir file:///tmp

mkdir: `file:///tmp': File exists


In [13]:
#View headers dataset
!head /content/wfpvam_foodprices.csv

In [14]:
%%writefile food_price_analysis.py
from mrjob.job import MRJob
import csv

class MRAvgPriceByYear(MRJob):

    def mapper(self, _, line):
        row = next(csv.reader([line]))
        if row[0] != "adm0_id":  # Skip header
            try:
                commodity = row[7]  # cm_name
                year = row[15]     # mp_year
                price = float(row[16])  # mp_price
                yield (commodity, year), (price, 1)
            except (ValueError, IndexError):
                pass  # Skip malformed rows

    def reducer(self, key, values):
        total = 0
        count = 0
        for price, cnt in values:
            total += price
            count += cnt
        yield key, round(total / count, 2)

if __name__ == '__main__':
    MRAvgPriceByYear.run()

Writing food_price_analysis.py


In [15]:
# Running the process
!python food_price_analysis.py /content/wfpvam_foodprices.csv > results.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/food_price_analysis.root.20250520.213709.590285
Running step 1 of 1...
job output is in /tmp/food_price_analysis.root.20250520.213709.590285/output
Streaming final output from /tmp/food_price_analysis.root.20250520.213709.590285/output...
Removing temp directory /tmp/food_price_analysis.root.20250520.213709.590285...


In [16]:
#Show result
!cat results.txt